<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/Seed_VC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Launch Seed-VC**
#@markdown This cell will clone the repository, modify the launch settings for a public URL, install all dependencies quietly, and start the Gradio app. Please wait for the installation and launch process to complete. A public URL (like `*.gradio.live`) will appear in the output below.

import os
import sys
import re # Import regex module for flexible replacement

print("⏳ Cloning repository...")
# Clone quietly
!git clone -q https://huggingface.co/spaces/Plachta/Seed-VC app
print("✅ Repository cloned.")

# Change directory
try:
    os.chdir('app')
    print(f"✅ Changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"❌ Error: 'app' directory not found after cloning. Check clone step.")
    sys.exit(1)

print("⏳ Modifying app.py for public sharing...")
# Modify app.py to add share=True using Python
try:
    # Try reading with utf-8, fallback to default encoding if needed
    try:
        with open('app.py', 'r', encoding='utf-8') as f:
            content = f.read()
    except UnicodeDecodeError:
        print("⚠️ Warning: Failed to read app.py as UTF-8, trying default system encoding.")
        with open('app.py', 'r') as f:
            content = f.read()

    # Use regex to find .launch() potentially with arguments inside and add share=True
    modified_content, n_replacements = re.subn(
        r"(\.launch\s*\()\s*([^)]*\)?)",  # Pattern: .launch( possibly_existing_args )
        r'\1share=True, \2',          # Replacement: .launch(share=True, possibly_existing_args )
        content,
        count=1                       # Replace likely the last relevant occurrence
    )

    if n_replacements > 0:
        # Try writing with utf-8, fallback to default encoding if needed
        try:
            with open('app.py', 'w', encoding='utf-8') as f:
                f.write(modified_content)
        except UnicodeEncodeError:
            print("⚠️ Warning: Failed to write app.py as UTF-8, trying default system encoding.")
            with open('app.py', 'w') as f:
                 f.write(modified_content)
        print("✅ app.py modified successfully to include share=True.")
        # Optional: verify the change
        # !grep "launch(share=True" app.py
    else:
        # Fallback or warning if the pattern wasn't found
        print("⚠️ Warning: Could not automatically find and modify '.launch()' using regex in app.py.")
        print("   Trying simple replacement as fallback...")
        fallback_success = False
        # Common variable names for Gradio interfaces
        for var_name in ['app', 'demo', 'iface', 'interface']:
             pattern_simple = f'{var_name}.launch()'
             if pattern_simple in content:
                 modified_content = content.replace(pattern_simple, f'{var_name}.launch(share=True)', 1)
                 try:
                     with open('app.py', 'w', encoding='utf-8') as f:
                        f.write(modified_content)
                 except UnicodeEncodeError:
                     with open('app.py', 'w') as f:
                        f.write(modified_content)
                 print(f"✅ app.py modified using simple replacement for '{pattern_simple}'.")
                 fallback_success = True
                 break # Stop after first successful simple replacement
        if not fallback_success:
             print("❌ Fallback replacement failed. Manual check of app.py might be needed if public URL is required.")
             # Proceeding anyway, might only get local URL

except FileNotFoundError:
    print(f"❌ Error: 'app.py' not found in the {os.getcwd()} directory.")
    sys.exit(1)
except Exception as e:
    print(f"❌ Error modifying app.py: {e}")
    # Decide if you want to exit or continue without the modification
    # sys.exit(1) # Or just print a warning and continue

print("⏳ Installing dependencies quietly... (This may take several minutes)")
# Install quietly from requirements.txt AND the specified packages, redirect output
# Pip handles duplicates gracefully if they are also in requirements.txt
!pip install --quiet -r requirements.txt protobuf tensorflow numpy gradio spaces > pip_install.log 2>&1

# Check if installation seems okay by checking for a common package (e.g., gradio or tensorflow)
install_ok = False
try:
    import gradio
    import tensorflow
    print("✅ Dependencies installed successfully (checked Gradio & TensorFlow).")
    install_ok = True
except ImportError as e:
    print(f"⚠️ Warning: Key dependency import failed ({e}). Installation might be incomplete.")
    print("   Check pip_install.log for details if errors occur during launch.")
    # !cat pip_install.log # Uncomment to view log if needed

# Optionally add a small delay if installs sometimes need a moment to settle (usually not needed)
# import time
# time.sleep(2)

print("\n🚀 Launching the Seed_VC app... Please wait for the public URL.")
# Run the app - Output will include the Gradio link
!python app.py